In [1]:
import Analysis
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
%matplotlib

Using matplotlib backend: TkAgg


In [54]:
def search(data, t):
    seq = data
    min = 0
    max = len(seq) - 1
    while True:        
        m = (min + max) // 2
        if max < min:
            if np.abs(t-seq[m]) < np.abs(t-seq[m-1]):          
                return m
            else:
                return m-1        
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m

In [2]:
from matplotlib.gridspec import GridSpec
mpl.rcParams.update({'font.size': 8})
gs = GridSpec(1000,1000)

In [3]:
def t(min,max):
    diff = max-min
    p = diff*.3
    return min-p,max+p

In [665]:
run_2014 = Analysis.generate_2014()

In [234]:
#Plot filenum 9, B = 5.5800 T

fig = plt.figure(figsize = [7, 2.5])
ax1 = plt.subplot(gs[:900,:])
run_2014.prune_data[9].plot()
run_2014.prune_data[9].fits[0].plot_select('l','k',2,-18.6613,-13.8051)
run_2014.prune_data[9].fits[0].plot_select('l','k--',2,-13.8051,-7.2823)

run_2014.prune_data[9].fits[0].plot_select('l','k',2,-7.2823,-5.9637)

run_2014.prune_data[9].fits[0].plot_select('r','r--',2,-18.6613,-13.8051)
run_2014.prune_data[9].fits[0].plot_select('r','r',2,-13.8051,-5.9637)

run_2014.prune_data[9].fits[2].plot_select('l','k',2,-20.4724,-20.3559)
run_2014.prune_data[9].fits[2].plot_select('r','r--',2,-20.4724,-20.3559)

run_2014.prune_data[9].fits[2].plot_select('l','k--',2,-20.3559,-24.3259)
run_2014.prune_data[9].fits[2].plot_select('r','r',2,-20.3559,-24.3259)

run_2014.prune_data[9].fits[2].plot_select('l','k',2,-24.3259,-24.4258)
run_2014.prune_data[9].fits[2].plot_select('r','r--',2,-24.3259,-24.4258)

run_2014.prune_data[9].fits[2].plot_select('l','k--',2,-24.4258,-25.0974)
run_2014.prune_data[9].fits[2].plot_select('r','r',2,-24.4258,-25.0974)

run_2014.prune_data[9].fits[2].plot_select('l','k',2,-25.0974,-25.2132)
run_2014.prune_data[9].fits[2].plot_select('r','r--',2,-25.0974,-25.2132)


run_2014.prune_data[9].fits[2].plot_select('r','r',2,-25.2132,-30)
run_2014.prune_data[9].fits[2].plot_select('l','k--',2,-25.2132,-30)

plt.xlim([-30,-4.1])

plt.ylabel('R$_{D}$ (k$\Omega$)',size=12)
plt.xlabel('V$_{P}$ (mV)',size=12)


In [235]:
plt.savefig('9 - 5580',dpi=1200)

In [57]:
from scipy.optimize import curve_fit

In [67]:
def func(x,freq,amp,phase_l,height,slope,v0):
    return height + amp * np.sin(freq * x + phase_l) + slope * (x - v0)
    

In [104]:
def gen_func(fit):
    def fitting_func(x, phase_r):
        return func(x,fit[0],fit[1],phase_r,fit[3],fit[4],fit[5])
    return fitting_func

In [144]:
def make_fit(popt,popt2,left,right):
    d_phi = ((popt2[0] - popt[2])/np.pi)%2
    return Analysis.Fit([popt[0],popt[1],popt2[0],popt[2],popt[3],popt[4],1,left,right,d_phi,1,popt[5]])

In [567]:
def make_fit_array(popt,popt2,left,right):
    d_phi = ((popt2[0] - popt[2])/np.pi)%2
    return np.array([popt[0],popt[1],popt2[0],popt[2],popt[3],popt[4],1,left,right,d_phi,1,popt[5]])

In [1]:
def data_partition(data_x,data_y,sections):
    x = list()
    y = list()
    for i in sections:
        i_min = search(data_x,i[0])
        i_max = search(data_x,i[1])+1
        x.extend(data_x[i_min:i_max])
        y.extend(data_y[i_min:i_max])
    return np.array(x), np.array(y)

In [340]:
filenum = 7

x_temp, y_temp = data_partition(run_2014.prune_data[filenum].x,run_2014.prune_data[filenum].y,[[-30.3898,-21.95],[-21.0302,-19.93]])

popt, pcov = curve_fit(func, x_temp, y_temp)

slip_func = gen_func(popt)

In [341]:
run_2014.prune_data[7].plot()
plt.plot(run_2014.prune_data[filenum].x[i_min:i_max+1000],func(run_2014.prune_data[filenum].x[i_min:i_max+1000],*popt),linewidth = 3, color = 'r')

In [330]:
i_min = search(run_2014.prune_data[filenum].x,-21.9504)
i_max = search(run_2014.prune_data[filenum].x,-21.0615)+1

temp = (popt[2]%(2*np.pi))
guess = temp - 0.25 * np.pi
popt2, pcov2 = curve_fit(slip_func, run_2014.prune_data[filenum].x[i_min:i_max], run_2014.prune_data[filenum].y[i_min:i_max],p0=[guess],bounds = ([-2*np.pi],[2*np.pi]))

In [331]:
run_2014.prune_data[7].plot()
plt.plot(run_2014.prune_data[filenum].x[i_min:i_max],slip_func(run_2014.prune_data[filenum].x[i_min:i_max],*popt2),linewidth = 3, color = 'r')

In [346]:
new_fit = make_fit(popt,popt2,-30.3898,-19.93)

In [353]:
run_2014.prune_data[7].plot()
new_fit.plot(False)

In [454]:
#Plot filenum 7, B = 5.581 T

filenum = 7
fig = plt.figure(figsize = [7, 2.5])
ax1 = plt.subplot(gs[:900,:])

plt.ylabel('R$_{D}$ (k$\Omega$)',size=12)
plt.xlabel('V$_{P}$ (mV)',size=12)


run_2014.prune_data[filenum].plot()

run_2014.prune_data[filenum].fits[3].plot_select('l','k--',2,-30.3754,-21.9616)
run_2014.prune_data[filenum].fits[3].plot_select('r','r',2,-30.3754,-21.9616)

run_2014.prune_data[filenum].fits[3].plot_select('l','k',2,-21.9616,-21.0567)
run_2014.prune_data[filenum].fits[3].plot_select('r','r--',2,-21.9616,-21.0567)

run_2014.prune_data[filenum].fits[3].plot_select('l','k--',2,-21.0567,-20.205)
run_2014.prune_data[filenum].fits[3].plot_select('r','r',2,-21.0567,-20.205)

run_2014.prune_data[filenum].fits[3].plot_select('l','k--',2,-21.0567,-20.205)
run_2014.prune_data[filenum].fits[3].plot_select('r','r',2,-21.0567,-20.205)

run_2014.prune_data[filenum].fits[0].plot_select('l','k',2,-19.0995,-13.9972)
run_2014.prune_data[filenum].fits[0].plot_select('r','r--',2,-19.0995,-13.9972)

run_2014.prune_data[filenum].fits[0].plot_select('l','k--',2,-13.9972,-12.5438)
run_2014.prune_data[filenum].fits[0].plot_select('r','r',2,-13.9972,-12.5438)

run_2014.prune_data[filenum].fits[0].plot_select('l','k',2,-12.5438,-12.4247)
run_2014.prune_data[filenum].fits[0].plot_select('r','r--',2,-12.5438,-12.4247)

run_2014.prune_data[filenum].fits[0].plot_select('l','k--',2,-12.4247,-8.2276)
run_2014.prune_data[filenum].fits[0].plot_select('r','r',2,-12.4247,-4)

run_2014.prune_data[filenum].fits[0].plot_select('l','k',2,-8.2276,-6.713)

plt.xlim(-30.41,-4.077)

(-30.41, -4.077)

In [455]:
plt.savefig('7 - 5581',dpi=1200)

In [460]:
#Plot filenum 7, B = 5.581 T version 2

filenum = 7
fig = plt.figure(figsize = [7, 2.5])
ax1 = plt.subplot(gs[:900,:])

plt.ylabel('R$_{D}$ (k$\Omega$)',size=12)
plt.xlabel('V$_{P}$ (mV)',size=12)


run_2014.prune_data[filenum].plot()

run_2014.prune_data[filenum].fits[3].plot_select('r','k',2,-30.3754,-21.9616)
run_2014.prune_data[filenum].fits[3].plot_select('l','r--',2,-30.3754,-21.9616)

run_2014.prune_data[filenum].fits[3].plot_select('r','k--',2,-21.9616,-21.0567)
run_2014.prune_data[filenum].fits[3].plot_select('l','r',2,-21.9616,-21.0567)

run_2014.prune_data[filenum].fits[3].plot_select('r','k',2,-21.0567,-20.205)
run_2014.prune_data[filenum].fits[3].plot_select('l','r--',2,-21.0567,-20.205)

run_2014.prune_data[filenum].fits[3].plot_select('r','k',2,-21.0567,-20.205)
run_2014.prune_data[filenum].fits[3].plot_select('l','r--',2,-21.0567,-20.205)

run_2014.prune_data[filenum].fits[0].plot_select('l','k',2,-19.0995,-13.9972)
run_2014.prune_data[filenum].fits[0].plot_select('r','r--',2,-19.0995,-13.9972)

run_2014.prune_data[filenum].fits[0].plot_select('l','k--',2,-13.9972,-12.5438)
run_2014.prune_data[filenum].fits[0].plot_select('r','r',2,-13.9972,-12.5438)

run_2014.prune_data[filenum].fits[0].plot_select('l','k',2,-12.5438,-12.4247)
run_2014.prune_data[filenum].fits[0].plot_select('r','r--',2,-12.5438,-12.4247)

run_2014.prune_data[filenum].fits[0].plot_select('l','k--',2,-12.4247,-8.2276)
run_2014.prune_data[filenum].fits[0].plot_select('r','r',2,-12.4247,-4)

run_2014.prune_data[filenum].fits[0].plot_select('l','k',2,-8.2276,-6.715)

plt.xlim(-30.41,-4.077)

(-30.41, -4.077)

In [458]:
plt.savefig('7 - 55815 v2',dpi=1200)

In [393]:
filenum = 7

x_temp, y_temp = data_partition(run_2014.prune_data[filenum].x,run_2014.prune_data[filenum].y,[[-21.9616,-21.0567],[-19.8056,-13.9972],[-12.5438,-12.4247],[-8.07224,-7.9235],[-7.5887,-7.457]])

guess = [run_2014.prune_data[filenum].fits[0].freq,run_2014.prune_data[filenum].fits[0].amp,run_2014.prune_data[filenum].fits[0].phase_l,run_2014.prune_data[filenum].fits[0].height,run_2014.prune_data[filenum].fits[0].slope,run_2014.prune_data[filenum].fits[0].v0]

popt, pcov = curve_fit(func, x_temp, y_temp, p0 = guess)

slip_func = gen_func(popt)

In [394]:
run_2014.prune_data[filenum].plot()
plt.plot(run_2014.prune_data[filenum].x,func(run_2014.prune_data[filenum].x,*popt),linewidth = 3, color = 'r')

In [395]:
x_temp, y_temp = data_partition(run_2014.prune_data[filenum].x,run_2014.prune_data[filenum].y,[[-13.8291,-12.559],[-12.4051,-8.254],[-7.8757,-7.7331],[-7.3923,-7.2712]])

temp = (popt[2]%(2*np.pi))
guess = temp - 0.25 * np.pi
popt2, pcov2 = curve_fit(slip_func, x_temp, y_temp,p0=[guess],bounds = ([-2*np.pi],[2*np.pi]))

In [397]:
run_2014.prune_data[filenum].plot()
plt.plot(run_2014.prune_data[filenum].x,slip_func(run_2014.prune_data[filenum].x,*popt2),linewidth = 3, color = 'r')

In [414]:
new_fit = make_fit(popt,popt2,-21.96,-5.15)

In [415]:
new_fit.phase_r, new_fit.phase_l = new_fit.phase_l, new_fit.phase_r

In [492]:
#Plot filenum 7, B = 5.5815 T version 3

filenum = 7
fig = plt.figure(figsize = [7, 2.5])
ax1 = plt.subplot(gs[:900,:])

plt.ylabel('R$_{D}$ (k$\Omega$)',size=12)
plt.xlabel('V$_{P}$ (mV)',size=12)


run_2014.prune_data[filenum].plot()

run_2014.prune_data[filenum].fits[3].plot_select('r','k',2,-30.3754,-21.9616)
run_2014.prune_data[filenum].fits[3].plot_select('l','r--',2,-30.3754,-21.9616)

run_2014.prune_data[filenum].fits[3].plot_select('r','k--',2,-21.9616,-21.0567)
run_2014.prune_data[filenum].fits[3].plot_select('l','r',2,-21.9616,-21.0567)

run_2014.prune_data[filenum].fits[3].plot_select('r','k',2,-21.0567,-20.205)
run_2014.prune_data[filenum].fits[3].plot_select('l','r--',2,-21.0567,-20.205)

run_2014.prune_data[filenum].fits[3].plot_select('r','k',2,-21.0567,-20.205)
run_2014.prune_data[filenum].fits[3].plot_select('l','r--',2,-21.0567,-20.205)

new_fit.plot_select('l','k',2,-19.0995,-13.9972)
new_fit.plot_select('r','r--',2,-19.0995,-13.9972)

new_fit.plot_select('l','k--',2,-13.9972,-12.5438)
new_fit.plot_select('r','r',2,-13.9972,-12.5438)

new_fit.plot_select('l','k',2,-12.5438,-12.4247)
new_fit.plot_select('r','r--',2,-12.5438,-12.4247)

new_fit.plot_select('l','k--',2,-12.4247,-8.2276)
new_fit.plot_select('r','r',2,-12.4247,-4)

new_fit.plot_select('l','k',2,-8.2276,-6.)

plt.xlim(-30.41,-4.077)

(-30.41, -4.077)

In [493]:
plt.savefig('7 - 5581 v3',dpi=1200)

In [605]:
#Plot filenum 4, B = 5.5825 T

filenum = 4
fig = plt.figure(figsize = [7, 2.5])
ax1 = plt.subplot(gs[:900,:])

plt.ylabel('R$_{D}$ (k$\Omega$)',size=12)
plt.xlabel('V$_{P}$ (mV)',size=12)


run_2014.prune_data[filenum].plot()

run_2014.prune_data[filenum].fits[3].plot_select('l','r--',2,-4.9934,-3.9992)
run_2014.prune_data[filenum].fits[3].plot_select('r','k',2,-4.9934,-3.9992)

run_2014.prune_data[filenum].fits[3].plot_select('l','r',2,-5.307,-4.9934)
run_2014.prune_data[filenum].fits[3].plot_select('r','k--',2,-5.307,-4.9934)

run_2014.prune_data[filenum].fits[3].plot_select('l','r--',2,-7.1903,-5.307)
run_2014.prune_data[filenum].fits[3].plot_select('r','k',2,-7.1903,-5.307)

run_2014.prune_data[filenum].fits[0].plot_select('l','k--',2,-14.3759,-7.1903)
run_2014.prune_data[filenum].fits[0].plot_select('r','r',2,-14.3759,-7.1903)

run_2014.prune_data[filenum].fits[0].plot_select('l','k',2,-20.4899,-14.3759)
run_2014.prune_data[filenum].fits[0].plot_select('r','r--',2,-20.4899,-14.3759)



plt.xlim(-31.03,-4.0177)

(-31.03, -4.0177)

In [606]:
plt.savefig('4 - 55825',dpi=1200)

In [675]:
#Plot filenum 0, B = 5.5845 T

filenum = 0
fig = plt.figure(figsize = [7, 2.5])
ax1 = plt.subplot(gs[:900,:])

plt.ylabel('R$_{D}$ (k$\Omega$)',size=12)
plt.xlabel('V$_{P}$ (mV)',size=12)


run_2014.prune_data[filenum].plot()

run_2014.prune_data[filenum].fits[1].plot_select('l','r',2,-22.151,-19.4295)
run_2014.prune_data[filenum].fits[1].plot_select('r','k--',2,-22.151,-19.4295)

run_2014.prune_data[filenum].fits[0].plot_select('l','k',2,-19.4295,-14.8558)
run_2014.prune_data[filenum].fits[0].plot_select('r','r--',2,-19.4295,-14.8558)

run_2014.prune_data[filenum].fits[0].plot_select('l','k--',2,-14.8558,-8.458)
run_2014.prune_data[filenum].fits[0].plot_select('r','r',2,-14.8558,-8.458)

plt.xlim(-31.016,-4.9)

(-31.016, -4.9)

In [671]:
plt.savefig('0 - 55845',dpi=1200)